# Modifs by Sacha 18/10

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [2]:
import os
from PIL import Image

In [3]:
# Sacha : Added 1. Batch norm + 2. Dropout
# Sacha : I don't know how to set the dropout ≠ 0 in the eval phase 
# (actually I have an idea, but it's not good to do it)



class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Encoder
        # 256 x 256 x 1
        self.encod1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=4, padding=1, stride=2),
            nn.LeakyReLU(),
            nn.BatchNorm2d(64)
        )
         # 128 x 128 x 64
        self.encod2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, padding=1, stride=2),
            nn.LeakyReLU(),
            nn.BatchNorm2d(128),
        )
        # 64 x 64 x 128
        self.encod3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, padding=1, stride=2),
            nn.LeakyReLU(),
            nn.BatchNorm2d(256),
        ) 
        # 32 x 32 x 256
        self.encod4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, padding=1, stride=2),
            nn.LeakyReLU(),
            nn.BatchNorm2d(512),
        )
        # 16 x 16 x 512
    
        # Decoder
        self.decod1 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(512),
        )
        # 16 x 16 x 512
        self.decod2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, padding=1, stride=2),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(256),
        )
        # 32 x 32 x 256
        self.decod3 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, padding=1, stride=2),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(128),
        )
        # 64 x 64 x 128
        self.decod4 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, padding=1, stride=2),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(64),
        )
        # 128 x 128 x 64
        self.decodout = nn.Sequential(
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, padding=1, stride=2),
            #256 x 256 x 3
            nn.Tanh())
     
    
    def forward(self, x):
        o1 = self.encod1(x)
        o2 = self.encod2(o1)
        o3 = self.encod3(o2)
        o4 = self.encod4(o3)
    
        o5 = self.decod1(o4) + o4
        o6 = self.decod2(o5) + o3
        o7 = self.decod3(o6) + o2 
        o8 = self.decod4(o7) + o1
        out = self.decodout(o8)
        
        return out

In [4]:
image = Image.open('/Users/sachaizadi/Desktop/man.jpeg')
image = np.array(image)[:,:,2]
image = torch.from_numpy(image)

In [5]:
image = image.type(torch.FloatTensor).resize_(2, 1, 416, 416)

In [6]:
image.size()

torch.Size([2, 1, 416, 416])

In [7]:
model = Generator()
model.train()
test = model(image)

In [8]:
test

tensor([[[[ 0.0302, -0.4196,  0.1739,  ..., -0.6910, -0.7610, -0.4488],
          [-0.0532, -0.2185,  0.7842,  ..., -0.7984, -0.3094, -0.3748],
          [-0.4358,  0.1778,  0.5143,  ...,  0.4868, -0.8784,  0.1371],
          ...,
          [-0.0319, -0.0505, -0.0532,  ..., -0.0065, -0.0686, -0.0468],
          [-0.0603, -0.0349, -0.0750,  ...,  0.0119, -0.0508, -0.0459],
          [ 0.0072, -0.0034, -0.0663,  ..., -0.0366, -0.0736,  0.0094]],

         [[ 0.5509, -0.3654,  0.0410,  ...,  0.4023, -0.0541,  0.1148],
          [-0.4612, -0.1464, -0.5600,  ..., -0.2965, -0.5570, -0.3555],
          [ 0.5735,  0.1425, -0.0369,  ...,  0.5969,  0.3961,  0.4859],
          ...,
          [ 0.0059,  0.0953, -0.0342,  ...,  0.0645, -0.0175,  0.0845],
          [ 0.0363,  0.0237,  0.0661,  ...,  0.0900,  0.0679, -0.0935],
          [ 0.0073,  0.0680,  0.0328,  ...,  0.0723,  0.0363,  0.0606]],

         [[-0.0615,  0.4077, -0.0556,  ...,  0.7786, -0.6226, -0.6409],
          [-0.2980,  0.7472, -

In [9]:
image = image.type(torch.FloatTensor).resize_(1, 1, 416, 416)
model.eval()
model(image)

tensor([[[[ 3.0902e-01, -5.1120e-01,  1.8097e-01,  ..., -7.6587e-01,
           -9.0539e-01, -3.1177e-01],
          [ 3.1504e-01, -7.2810e-01,  9.9943e-01,  ..., -9.1368e-01,
            6.4699e-01, -8.8769e-01],
          [-3.8577e-01,  9.4852e-03,  9.1213e-01,  ..., -2.5096e-01,
           -7.8961e-01,  7.9981e-01],
          ...,
          [-2.8085e-02, -3.2910e-02, -2.9058e-02,  ..., -3.2692e-02,
           -2.8857e-02, -3.8452e-02],
          [-3.8178e-02,  7.3773e-03, -1.9104e-02,  ...,  7.2986e-03,
           -1.9531e-02, -1.5214e-02],
          [-1.9346e-02, -2.0680e-02, -1.9141e-02,  ..., -2.0522e-02,
           -1.9059e-02, -2.6532e-02]],

         [[ 8.0624e-01, -1.3629e-01, -2.5619e-01,  ...,  1.3658e-01,
            5.5926e-02, -2.4281e-01],
          [-9.0311e-01, -3.6824e-01, -8.9633e-01,  ..., -6.6439e-01,
           -5.0524e-01, -8.9193e-01],
          [ 8.8117e-01,  4.1550e-01, -5.5284e-01,  ...,  7.5034e-01,
           -2.7141e-02,  7.1162e-01],
          ...,
     

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv= nn.Sequential(
           # 256 x 256 x 3

            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(32),
            

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(64),
            
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.LeakyReLU(),  
            nn.BatchNorm2d(128),
            
            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(256),
            
            
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ELU(),
            # 256 x 256 x 512
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features = 256*256*512, out_features = 1 ),
        )
      
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 256*256*512)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [11]:
image = Image.open('/Users/sachaizadi/Desktop/man.jpeg')
image = np.array(image)[:,:,]
image = torch.from_numpy(image)
image = image.type(torch.FloatTensor).resize_(2, 3, 256, 256)

In [12]:
model = Discriminator()
model.train()
test = model(image)
print(test)
model.eval()
test = model(image)
print(test)

/usr/local/lib/python3.7/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.4993],
        [0.4993]], grad_fn=<SigmoidBackward>)
tensor([[0.4995],
        [0.4995]], grad_fn=<SigmoidBackward>)


In [ ]:
class GAN(object):
    def __init__(self, config):      
        self.config = config
        self.data_init()
        self.model_init()
        
    def data_init(self):
        self.dataset = Dataset_Color()
        self.training_data_loader = DataLoader(dataset= self.dataset, batch_size= self.config.batch_size, shuffle=True)
        
    def model_init(self):
        self.dis_model = Discriminator()
        self.dis_model = self.dis_model.cuda()
        self.gen_model = Generator()
        self.gen_model = self.gen_model.cuda()
        self.loss = nn.BCELoss(size_average=False).cuda()
        self.optimizer_dis = optim.Adam(self.dis_model.parameters(),lr= self.config.lr_dis,betas=self.config.betas)
        self.optimizer_gen = optim.Adam(self.gen_model.parameters(),lr= self.config.lr_gen,betas=self.config.betas)
        
    def train(self):
        return self.training( self.dis_model, self.gen_model, self.training_data_loader, self.loss, self.optimizer_dis, self.optimizer_gen,n_epochs = self.config.n_epochs)
      
        
    def training(self,dis_model, gen_model, data_loader, loss_fn, 
              dis_optimizer, gen_optimizer, n_epochs= 1):
        
        use_gpu =  torch.cuda.is_available()
        if use_gpu == True:
            torch.cuda.set_device(0)


        dis_model.train(True)
        gen_model.train(True)

        dis_loss = np.zeros(n_epochs)
        gen_loss = np.zeros(n_epochs)

        for epoch_num in range(n_epochs):

            dis_running_loss = 0.0
            gen_running_loss = 0.0
            size = 0

            for data in data_loader:

                clr_img, bw_img = data['clr'], data['bw']

                if use_gpu == True:
                    clr_img = clr_img.cuda()
                    bw_img  = bw_img.cuda()

                batch_size = clr_img.size(0)
                size += batch_size
                # get image size            
                # reset optimizers
                dis_optimizer.zero_grad()
                gen_optimizer.zero_grad()

                # TRAIN THE DISCRIMINATOR
                out = dis_model(clr_img)

                loss = loss_fn(out, torch.ones(batch_size, 1).cuda()) # check size of torch.ones
                loss.backward()
                dis_running_loss += loss.data.cpu().numpy()

                fake_img = gen_model(bw_img).detach()
                fake_out = dis_model(fake_img)
                loss = loss_fn(fake_out, torch.zeros(batch_size, 1).cuda()) # check size of torch.zeros
                dis_running_loss += loss.data.cpu().numpy()
                loss.backward()
                dis_optimizer.step()


                fake_img = gen_model(bw_img).detach()
                fake_out = dis_model(fake_img)
                loss = loss_fn(fake_out, torch.ones(batch_size, 1).cuda())
                gen_running_loss += loss.data.cpu().numpy()
                loss.backward()
                gen_optimizer.step() #Added by Sacha




            epoch_dis_loss = dis_running_loss / size
            epoch_gen_loss = gen_running_loss / size
            dis_loss[epoch_num] = epoch_dis_loss
            gen_loss[epoch_num] = epoch_gen_loss

            if (epoch_num+1) % 50 == 0 and epoch_num!=0: 
                print('Train - Discriminator Loss: {:.4f} Generator Loss: {:.4f}'.format(epoch_dis_loss, epoch_gen_loss))
                
            if (epoch_num+1) % self.config.save_frequency == 0 and epoch_num !=0: 
                if not os.path.exists(self.config.model_dir):
                    os.makedirs(self.config.model_dir)
                state = {'epoch': epoch_num, 'gen_model_state_dict': gen_model.state_dict(), 'dis_model_state_dict': dis_model.state_dict(), 'dis_optimizer': dis_optimizer, 'gen_optimizer': gen_optimizer}
                torch.save(state, self.config.model_dir + 'model_weights_epoch_%i.pk' %epoch_num)
                print ("Saved Model") 